# 🧮 Herramientas Financieras - Tests

Probamos cada herramienta individualmente antes de integrar con el agente.

## 📦 Package instll


In [86]:
!pip install -qU langchain langgraph langchain-openai langchain_community psycopg[binary,pool]==3.2.6 langchain_experimental numpy-financial


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🛠️ Imports

In [ ]:
import os

# Opción 1: Hardcoded (para testing)
os.environ["ANTHROPIC_API_KEY"] = 
# Opción 2: Desde archivo (recomendado)
# with open("anthropic_key.txt") as f:
#     os.environ["ANTHROPIC_API_KEY"] = f.read().strip()

In [110]:
llm = ChatAnthropic(model="claude-sonnet-4-20250514",temperature=0.15)
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage
from typing import TypedDict, Annotated, Literal  # <-- ¡Añade 'Literal' aquí!
import operator
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from langchain_anthropic import ChatAnthropic  
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda



In [111]:
class BonoInput(BaseModel):
    """Schema para calcular el valor presente de un bono."""
    valor_nominal: float = Field(description="Valor nominal (facial) del bono", gt=0)
    tasa_cupon_anual: float = Field(description="Tasa de interés del cupón **ANUAL** en % (ej. 6 para 6%)", ge=0, le=100)
    tasa_descuento_anual: float = Field(description="Tasa de descuento de mercado **ANUAL** (YTM) en % (ej. 5 para 5%)", ge=0, le=100)
    num_anos: int = Field(description="Número total de **AÑOS** hasta el vencimiento (ej. 5 para 5 años)", gt=0) # <-- Renombrado y aclarado
    frecuencia_cupon: int = Field(description="Pagos de cupón por año (ej. 1 para anual, 2 para semestral)", gt=0)

class VANInput(BaseModel):
    """Schema para calcular el Valor Actual Neto (VAN) de un proyecto."""
    tasa_descuento: float = Field(description="Tasa de descuento (WACC, TMAR) en %", ge=0, le=100)
    inversion_inicial: float = Field(description="Desembolso inicial como un número **POSITIVO** (ej. 100000)", gt=0) # <-- Corregido a POSITIVO
    flujos_caja: list[float] = Field(description="Lista de flujos de caja futuros (ej. [25000, 30000, 35000])") # <-- El LLM DEBE pasar una lista

    
class OpcionCallInput(BaseModel):
    """Schema para calcular opción call."""
    precio_actual: float = Field(description="Precio actual del activo en dólares", gt=0)
    precio_ejercicio: float = Field(description="Precio de ejercicio en dólares", gt=0)
    tiempo_vencimiento: float = Field(description="Tiempo hasta vencimiento en años", gt=0)
    tasa_libre_riesgo: float = Field(description="Tasa libre de riesgo en %", ge=0, le=100)
    volatilidad: float = Field(description="Volatilidad anual en %", gt=0, le=200)
    
    model_config = {
        "json_schema_extra": {
            "examples": [{"precio_actual": 100, "precio_ejercicio": 105, "tiempo_vencimiento": 0.5, "tasa_libre_riesgo": 5, "volatilidad": 20}]
        }
    }
class WACCInput(BaseModel):
    """Schema para calcular el Costo Promedio Ponderado de Capital (WACC)."""
    tasa_impuestos: float = Field(description="Tasa de impuestos corporativos en %", ge=0, le=100)
    costo_deuda: float = Field(description="Costo de la deuda (tasa de interés) en %", ge=0, le=100)
    costo_equity: float = Field(description="Costo del equity (capital propio) en %", ge=0, le=100)
    valor_mercado_deuda: float = Field(description="Valor de mercado total de la deuda en dólares", gt=0)
    valor_mercado_equity: float = Field(description="Valor de mercado total del equity (capital) en dólares", gt=0)

class CAPMInput(BaseModel):
    """Schema para calcular el Costo del Equity usando CAPM."""
    tasa_libre_riesgo: float = Field(description="Tasa libre de riesgo (ej. bonos del tesoro) en %", ge=0, le=100)
    beta: float = Field(description="Beta del activo (medida de volatilidad)", gt=0)
    retorno_mercado: float = Field(description="Retorno esperado del mercado (ej. S&P 500) en %", ge=0, le=100)

class SharpeRatioInput(BaseModel):
    """Schema para calcular el Ratio de Sharpe de un portafolio."""
    retorno_portafolio: float = Field(description="Retorno esperado del portafolio en %", ge=0, le=100)
    tasa_libre_riesgo: float = Field(description="Tasa libre de riesgo en %", ge=0, le=100)
    std_dev_portafolio: float = Field(description="Desviación estándar (volatilidad) del portafolio en %", gt=0, le=200)

class GordonGrowthInput(BaseModel):
    """Schema para calcular el valor de una acción usando el Modelo de Crecimiento de Gordon (DDM)."""
    dividendo_prox_periodo: float = Field(description="Dividendo esperado en el próximo periodo (D1) en dólares", gt=0)
    tasa_descuento_equity: float = Field(description="Tasa de descuento o costo del equity (Ke) en %", gt=0, le=100)
    tasa_crecimiento_dividendos: float = Field(description="Tasa de crecimiento constante de los dividendos (g) en %", ge=0, le=100)

print("✅ Pydantic Models definidos")

✅ Pydantic Models definidos


In [112]:
import numpy as np
import numpy_financial as npf
from scipy.stats import norm


@tool("calcular_valor_bono", args_schema=BonoInput)
def _calcular_valor_presente_bono(valor_nominal: float, tasa_cupon_anual: float, tasa_descuento_anual: float, num_anos: int, frecuencia_cupon: int) -> dict:
    """Calcula el valor presente de un bono."""
    try:
        # Usamos los nombres de argumentos correctos
        tasa_cupon_periodo = (tasa_cupon_anual / 100) / frecuencia_cupon
        tasa_descuento_periodo = (tasa_descuento_anual / 100) / frecuencia_cupon
        num_periodos_totales = num_anos * frecuencia_cupon # <--- Corregido
        
        pago_cupon = valor_nominal * tasa_cupon_periodo
        
        # Esta lógica es más robusta para el PV de un bono
        flujos_cupones = [pago_cupon] * num_periodos_totales
        valor_presente_cupones = npf.npv(tasa_descuento_periodo, flujos_cupones)
        valor_presente_nominal = valor_nominal / ((1 + tasa_descuento_periodo) ** num_periodos_totales)
        valor_bono = valor_presente_cupones + valor_presente_nominal

        return {"valor_presente_bono": round(valor_bono, 2)}
    except Exception as e:
        return {"error": str(e)}

@tool("calcular_van", args_schema=VANInput)
def _calcular_van(tasa_descuento: float, inversion_inicial: float, flujos_caja: list[float]) -> dict:
    """Calcula el Valor Actual Neto (VAN) de un proyecto."""
    try:
        tasa = tasa_descuento / 100
        # Arreglamos el bug: la inversión inicial debe ser negativa
        flujos_totales = [-abs(inversion_inicial)] + flujos_caja 
        van = npf.npv(tasa, flujos_totales)
        return {"van": round(van, 2), "interpretacion": "Si VAN > 0, el proyecto es rentable."}
    except Exception as e:
        return {"error": str(e)}
@tool("calcular_opcion_call", args_schema=OpcionCallInput)
def _calcular_valor_opcion_call(S: float, K: float, T: float, r: float, sigma: float) -> dict:
    """Calcula el valor de una Opción Call Europea usando Black-Scholes."""
    try:
        r_dec = r / 100
        sigma_dec = sigma / 100
        d1 = (np.log(S / K) + (r_dec + 0.5 * sigma_dec**2) * T) / (sigma_dec * np.sqrt(T))
        d2 = d1 - sigma_dec * np.sqrt(T)
        call_price = (S * norm.cdf(d1) - K * np.exp(-r_dec * T) * norm.cdf(d2))
        return {"valor_opcion_call": round(call_price, 4)}
    except Exception as e:
        return {"error": str(e)}

@tool("calcular_wacc", args_schema=WACCInput)
def _calcular_wacc(tasa_impuestos: float, costo_deuda: float, costo_equity: float, valor_mercado_deuda: float, valor_mercado_equity: float) -> dict:
    """Calcula el Costo Promedio Ponderado de Capital (WACC)."""
    try:
        t_c = tasa_impuestos / 100
        k_d = costo_deuda / 100
        k_e = costo_equity / 100
        D = valor_mercado_deuda
        E = valor_mercado_equity
        V = D + E
        wacc = (E / V) * k_e + (D / V) * k_d * (1 - t_c)
        return {"wacc_porcentaje": round(wacc * 100, 4)}
    except Exception as e:
        return {"error": str(e)}

@tool("calcular_capm", args_schema=CAPMInput)
def _calcular_capm(tasa_libre_riesgo: float, beta: float, retorno_mercado: float) -> dict:
    """Calcula el Costo del Equity (Ke) usando el Capital Asset Pricing Model (CAPM)."""
    try:
        rf = tasa_libre_riesgo / 100
        rm = retorno_mercado / 100
        k_e = rf + beta * (rm - rf)
        return {"costo_equity_porcentaje": round(k_e * 100, 4)}
    except Exception as e:
        return {"error": str(e)}

@tool("calcular_sharpe_ratio", args_schema=SharpeRatioInput)
def _calcular_sharpe_ratio(retorno_portafolio: float, tasa_libre_riesgo: float, std_dev_portafolio: float) -> dict:
    """Calcula el Ratio de Sharpe para medir el retorno ajustado al riesgo."""
    try:
        r_p = retorno_portafolio / 100
        r_f = tasa_libre_riesgo / 100
        std_p = std_dev_portafolio / 100
        sharpe = (r_p - r_f) / std_p
        return {"sharpe_ratio": round(sharpe, 4)}
    except Exception as e:
        return {"error": str(e)}

@tool("calcular_gordon_growth", args_schema=GordonGrowthInput)
def _calcular_gordon_growth(dividendo_prox_periodo: float, tasa_descuento_equity: float, tasa_crecimiento_dividendos: float) -> dict:
    """Calcula el valor de una acción usando el Modelo de Crecimiento de Gordon (DDM)."""
    try:
        D1 = dividendo_prox_periodo
        Ke = tasa_descuento_equity / 100
        g = tasa_crecimiento_dividendos / 100
        if Ke <= g:
            return {"error": "La tasa de descuento (Ke) debe ser mayor que la tasa de crecimiento (g)."}
        valor_accion = D1 / (Ke - g)
        return {"valor_intrinseco_accion": round(valor_accion, 2)}
    except Exception as e:
        return {"error": str(e)}

In [113]:
# --- PASO 3: Creación de los Agentes Especialistas (Corregido con Prompts Estrictos Generalizados) ---

messages_placeholder = MessagesPlaceholder(variable_name="messages")

def crear_agente_especialista(llm, tools, system_prompt):
    """Función helper para crear un agente con un prompt de sistema."""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        messages_placeholder,
    ])
    return create_react_agent(llm, tools, prompt=prompt)

# --- Agente 1: Analista de Renta Fija ---
tools_renta_fija = [_calcular_valor_presente_bono]
agent_renta_fija = crear_agente_especialista(
    llm, 
    tools_renta_fija, 
    system_prompt="""Eres un especialista en Renta Fija.
Tu único trabajo es usar SÓLO tu herramienta 'calcular_valor_bono'.
**NUNCA respondas usando tu conocimiento general.**
Revisa cuidadosamente el historial de mensajes por si necesitas información previa.
Extrae los parámetros necesarios de la solicitud o del historial y llama a tu herramienta.
Si te piden algo que no puedes hacer con tu herramienta, di "No es mi especialidad, devuelvo al supervisor."."""
)

# --- Agente 2: Analista de Finanzas Corporativas ---
tools_fin_corp = [_calcular_van, _calcular_wacc]
agent_fin_corp = crear_agente_especialista(
    llm, 
    tools_fin_corp,
    system_prompt="""Eres un especialista en Finanzas Corporativas.
Tu trabajo es usar SÓLO tus herramientas 'calcular_van' y 'calcular_wacc'.
**NUNCA respondas usando tu conocimiento general.**
Revisa cuidadosamente el historial de mensajes por si necesitas información previa (ej. WACC calculado).
Extrae los parámetros necesarios de la solicitud o del historial y llama a la herramienta adecuada.
Si te piden algo que no puedes hacer con tus herramientas, di "No es mi especialidad, devuelvo al supervisor."."""
)

# --- Agente 3: Analista de Equity (Acciones) ---
tools_equity = [_calcular_gordon_growth]
agent_equity = crear_agente_especialista(
    llm, 
    tools_equity,
    system_prompt="""Eres un especialista en valoración de acciones (Equity).
Tu único trabajo es usar SÓLO tu herramienta 'calcular_gordon_growth'.
**NUNCA respondas usando tu conocimiento general.**
Revisa cuidadosamente el historial de mensajes. Si una tarea anterior calculó un valor necesario (como Ke o tasa de descuento), usa ESE valor.
Extrae el 'dividendo_prox_periodo' (D1), la 'tasa_descuento_equity' (Ke) y la 'tasa_crecimiento_dividendos' (g) de la solicitud del usuario o del historial.
Llama a tu herramienta con estos 3 parámetros.
Si no puedes encontrar los 3 parámetros, di "Faltan parámetros, devuelvo al supervisor."."""
)

# --- Agente 4: Analista de Portafolios ---
tools_portafolio = [_calcular_capm, _calcular_sharpe_ratio]
agent_portafolio = crear_agente_especialista(
    llm, 
    tools_portafolio,
    system_prompt="""Eres un especialista en Gestión de Portafolios.
Tu trabajo es usar SÓLO tus herramientas 'calcular_capm' y 'calcular_sharpe_ratio'.
**NUNCA respondas usando tu conocimiento general.**
Revisa cuidadosamente el historial de mensajes por si necesitas información previa.
Extrae los parámetros necesarios de la solicitud o del historial y llama a la herramienta adecuada.
Si te piden una tarea para la que no tienes herramienta (como 'calcular_gordon_growth'), **NO respondas a esa parte**.
Responde SÓLO la parte que SÍ puedes hacer con tus herramientas.
Luego, di "Tarea parcial completada, devuelvo al supervisor."."""
)

# --- Agente 5: Analista de Derivados ---
tools_derivados = [_calcular_valor_opcion_call]
agent_derivados = crear_agente_especialista(
    llm, 
    tools_derivados,
    system_prompt="""Eres un especialista en instrumentos derivados.
Tu único trabajo es usar SÓLO tu herramienta 'calcular_valor_opcion_call'.
**NUNCA respondas usando tu conocimiento general.**
Revisa cuidadosamente el historial de mensajes por si necesitas información previa.
Extrae los parámetros necesarios (S, K, T, r, sigma) de la solicitud o del historial y llama a tu herramienta.
Si te piden algo que no puedes hacer con tu herramienta, di "No es mi especialidad, devuelvo al supervisor."."""
)

print("✅ Agentes especialistas creados con prompts estrictos.")

✅ Agentes especialistas creados con prompts estrictos.


C:\Users\fguerrero\AppData\Local\Temp\ipykernel_3376\2625149408.py:12: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  return create_react_agent(llm, tools, prompt=prompt)


In [ ]:
# --- PASO 4 (Revisado): Supervisor y Estado con CIRCUIT BREAKER ---

class AgentState(TypedDict):
    messages: Annotated[list, lambda x, y: x + y]
    next_node: str
    # NUEVO CAMPO PARA EL CIRCUIT BREAKER:
    error_count: int 

class RouterSchema(BaseModel):
    """Elige el siguiente agente a llamar o finaliza la conversación."""
    next_agent: Literal[
        "Agente_Renta_Fija",
        "Agente_Finanzas_Corp",
        "Agente_Equity",
        "Agente_Portafolio",
        "Agente_Derivados",
        "FINISH"
    ] = Field(description="El nombre del agente especialista a quien enrutar la tarea. Elige 'FINISH' si la solicitud del usuario ha sido completamente respondida.")

supervisor_llm = llm.with_structured_output(RouterSchema)

supervisor_system_prompt = """Eres un supervisor de un equipo de analistas financieros.
Tu trabajo es recibir una solicitud del usuario y enrutarla al especialista correcto.
No intentes responder la pregunta tú mismo. Solo elige el siguiente paso.

El equipo de especialistas es:
1.  **Agente_Renta_Fija**: Tareas: Valoración de bonos (`calcular_valor_bono`).
2.  **Agente_Finanzas_Corp**: Tareas: Valoración de proyectos (VAN) y costo de capital (WACC).
3.  **Agente_Equity**: Tareas: Valoración de acciones (`calcular_gordon_growth`).
4.  **Agente_Portafolio**: Tareas: Calcular costo de equity (CAPM) y ratios de portafolio (Sharpe).
5.  **Agente_Derivados**: Tareas: Valoración de opciones (`calcular_opcion_call`).

Basado en la conversación (especialmente el último mensaje del usuario), decide a qué agente enrutar la tarea.
Si la solicitud ya fue respondida por un agente y no hay más pasos, elige 'FINISH'.
"""

# Definimos un límite de reintentos
MAX_RETRIES = 2

def supervisor_node(state: AgentState):
    """Nodo del supervisor que decide el siguiente paso."""
    print("--- SUPERVISOR (con Circuit Breaker v2) ---")
    
    # --- LÓGICA DEL CIRCUIT BREAKER ---
    error_count = state.get('error_count', 0)
    
    if state['messages']:
        last_message = state['messages'][-1]
        
        if (
            isinstance(last_message, AIMessage) and 
            not last_message.tool_calls
        ):
            # --- NUEVA LÓGICA PARA LEER EL CONTENIDO ---
            full_content = ""
            if isinstance(last_message.content, str):
                full_content = last_message.content.lower()
            elif isinstance(last_message.content, list):
                # Concatenar todos los 'text' parts de la lista
                for part in last_message.content:
                    if isinstance(part, dict) and 'text' in part:
                        full_content += part['text'].lower()
                    elif isinstance(part, str):
                        full_content += part.lower()
            # --- FIN LÓGICA NUEVA ---

            # Ahora hacemos el check en el string completo
            if ("problema técnico" in full_content or 
                "no puedo calcular" in full_content or
                "error" in full_content):
                
                error_count += 1
                print(f"Detectada falla de agente. Nuevo conteo de reintentos: {error_count}")

    # Comprobar si superamos el límite de reintentos
    if error_count >= MAX_RETRIES:
        print(f"Límite de reintentos ({MAX_RETRIES}) superado. Forzando finalización.")
        return {
            "messages": [AIMessage(content=f"Error: No se pudo completar la tarea después de {MAX_RETRIES} intentos. El bucle ha sido detenido.")],
            "next_node": "FINISH",
            "error_count": error_count 
        }
    # --- FIN DE LA LÓGICA ---

    # Enrutamiento normal
    messages = [HumanMessage(content=supervisor_system_prompt)] + state['messages']
    route = supervisor_llm.invoke(messages)
    print(f"Ruta decidida: {route.next_agent}")
    
    # Resetear el conteo
    previous_route = state.get('next_node', None)
    if route.next_agent == "FINISH" or route.next_agent != previous_route:
        error_count = 0 
    
    return {
        "next_node": route.next_agent, 
        "error_count": error_count
    }

In [115]:
# --- Construcción del Grafo ---
workflow = StateGraph(AgentState)

# 1. Añadir el nodo Supervisor
workflow.add_node("Supervisor", supervisor_node)

# 2. Añadir los nodos de los agentes especialistas
workflow.add_node("Agente_Renta_Fija", agent_renta_fija)
workflow.add_node("Agente_Finanzas_Corp", agent_fin_corp)
workflow.add_node("Agente_Equity", agent_equity)
workflow.add_node("Agente_Portafolio", agent_portafolio)
workflow.add_node("Agente_Derivados", agent_derivados)

# 3. Definir el Punto de Entrada
workflow.set_entry_point("Supervisor")

# 4. Definir el Enrutamiento Condicional
def conditional_router(state: AgentState):
    """Decide a qué nodo ir después del Supervisor."""
    return state["next_node"]

workflow.add_conditional_edges(
    "Supervisor",
    conditional_router,
    {
        "Agente_Renta_Fija": "Agente_Renta_Fija",
        "Agente_Finanzas_Corp": "Agente_Finanzas_Corp",
        "Agente_Equity": "Agente_Equity",
        "Agente_Portafolio": "Agente_Portafolio",
        "Agente_Derivados": "Agente_Derivados",
        "FINISH": END
    }
)

# 5. Definir las Aristas de Retorno
# ¡Importante! Después de que un especialista termine, debe volver al supervisor
# para que decida el siguiente paso (o si debe terminar).
workflow.add_edge("Agente_Renta_Fija", "Supervisor")
workflow.add_edge("Agente_Finanzas_Corp", "Supervisor")
workflow.add_edge("Agente_Equity", "Supervisor")
workflow.add_edge("Agente_Portafolio", "Supervisor")
workflow.add_edge("Agente_Derivados", "Supervisor")

# 6. Compilar el Grafo
# Añadimos una memoria para que recuerde la conversación
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [117]:
# --- PASO 6: Pruebas Simplificadas para MVP ---

# --- Prueba 1: VAN (Agente Finanzas Corp) ---
print("--- INICIO PRUEBA 1 (VAN) ---")
config = {"configurable": {"thread_id": "prueba-mvp-1"}}
# Prompt claro y directo para VAN
pregunta1 = "Calcula el VAN: inversión 20000, flujos [6000, 7000, 8000, 9000], tasa 8%" 

for event in graph.stream({"messages": [HumanMessage(content=pregunta1)]}, config=config):
    print(event)
    print("--------------------")

# --- Prueba 2: CAPM (Agente Portafolio) ---
print("\n\n--- INICIO PRUEBA 2 (CAPM) ---")
config = {"configurable": {"thread_id": "prueba-mvp-2"}}
# Prompt directo solo para CAPM
pregunta2 = """¿Cuál es el costo de equity (Ke) si la tasa libre de riesgo es 3%, 
   la beta es 1.2 y el retorno de mercado es 10%?"""

for event in graph.stream({"messages": [HumanMessage(content=pregunta2)]}, config=config):
    print(event)
    print("--------------------")

# --- Prueba 3: Bono (Agente Renta Fija) ---
print("\n\n--- INICIO PRUEBA 3 (Bono) ---")
config = {"configurable": {"thread_id": "prueba-mvp-3"}}
# Prompt directo para Bono
pregunta3 = "Tengo un bono nominal de $1000 a 5 años. Paga cupón semestral del 6% anual. El YTM de mercado es 5% anual. ¿Cuánto vale?"

for event in graph.stream({"messages": [HumanMessage(content=pregunta3)]}, config=config):
    print(event)
    print("--------------------")

# --- Prueba 4: Gordon Growth (Agente Equity) ---
print("\n\n--- INICIO PRUEBA 4 (Gordon Growth) ---")
config = {"configurable": {"thread_id": "prueba-mvp-4"}}
# Prompt directo para Gordon Growth, incluyendo Ke
pregunta4 = """Calcula el valor de una acción usando Gordon Growth:
   Dividendo próximo año (D1) = $2
   Costo de Equity (Ke) = 11.4%
   Tasa de crecimiento (g) = 5%"""

for event in graph.stream({"messages": [HumanMessage(content=pregunta4)]}, config=config):
    print(event)
    print("--------------------")

--- INICIO PRUEBA 1 (VAN) ---
--- SUPERVISOR (con Circuit Breaker v2) ---
Ruta decidida: Agente_Finanzas_Corp
{'Supervisor': {'next_node': 'Agente_Finanzas_Corp', 'error_count': 0}}
--------------------
{'Agente_Finanzas_Corp': {'messages': [HumanMessage(content='Calcula el VAN: inversión 20000, flujos [6000, 7000, 8000, 9000], tasa 8%', additional_kwargs={}, response_metadata={}, id='d1c27d72-dc0b-4b64-90d5-3f0d3afef80c'), AIMessage(content=[{'id': 'toolu_01DbmacDGgenFFjNfDNRvbf2', 'input': {'inversion_inicial': 20000, 'flujos_caja': [6000, 7000, 8000, 9000], 'tasa_descuento': 8}, 'name': 'calcular_van', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_016EYVzeocUkf2Ey3jvtxvMS', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1083, 'output_tokens': 11